## Monolingual Norec experiments with bert
Targeted sentiment analysis with simpletransformers for sequence tagging. Highly recommend to do this in a dedicated Python environment. You need PyTorch to interact with Cuda, and Simpletransformers to interact with pytorch, and you need the right Python version to support this chain. I suggest you begin with having a Cuda version that is listed in the pytorch installation guide, and take it from there.  

### GPU Clean installsimpletransformers.
For exploring norBERT with SimpleTransformers

In [7]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
# import nltk
import re
# from nltk.tokenize.simple import SpaceTokenizer
from helpers import *

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: True


## Prepare the data.
I collect the experiments to run as a list of tuples. Each experiment consists of training set and testing set, and a list of tags. This list of tags is used only for checking that the files have the tags we expect. Highly recommend to do this checking, that we have the expected tags in the datasets. Token and tags need to be spaceseparated

In [8]:
# Origianlly made for multiple datasets
datasets = [('data/spaceseparated/norec_fine/norec_fine_train.conll', 'data/spaceseparated/norec_fine/norec_fine_dev.conll', ['I-targ-negative', 'I-targ-positive',
 'B-targ-negative', 'B-targ-positive', 'O'])]
train_path, dev_path, tags = datasets[0]
# Check that the tags are consistent 
for path in [dev_path,train_path]:
    with open (path) as rf:
        text = rf.read()
        assert tagsset(text, separator = " ") == set(tags), tagsset(text, separator = " ")
print("Tags checked OK")

Tags checked OK


## Perform the fine-tuning
Note that this script does not automatically  create the folders needed to save the model and to record the output. I recommend you run the following cell with 1 epoch to see that this works, befor setting it back to 8 or whatever you consider to be adequate. 3 should be enough, but I got a litte better result with 8 so I kept that.

Simpletransformers has included the code for wandb but I have not tried to connect and use that what is supposed to be a great reporting and logging tool.

Note that if you run many epochs and save the models, you will need a lot of space.

In [9]:
# Run bert multilingual with the data from previous cell

family = "bert"
transformersmodels = ['bert-base-multilingual-cased' , 'NbAiLab/nb-bert-base',  'ltgoslo/norbert' ]
results = []

for transformersmodel in transformersmodels:
    model_args = NERArgs() 
    model_args.train_batch_size = 12
    model_args.num_train_epochs = 3
    model_args.weight_decay = 0.001
    model_args.overwrite_output_dir = True
    model_args.silent = False
    model_args.save_model_every_epoch = False
    model_args.save_eval_checkpoints = True
    model_args.save_steps = -1
    model_args.warmup_ratio = 0

    model = NERModel(family,transformersmodel , labels = tags,args=model_args)

    out_d = "outputs/simpletransformers/"+transformersmodel+"_"+train_path.split("/")[-2]

    model.train_model(train_path, output_dir= out_d)
    print(transformersmodel, "Done training")

    result, model_outputs, predictions = model.eval_model(dev_path)
    result["transformer_model"] = transformersmodel
    results.append(result)
    for k, v in result.items():
        print(transformersmodel, k, v)
    predictions, raw_outputs = model.predict (["Mannen på scenen synger stygt", "Damen på scenen synger stygt" , "Disse bilene har et fantastisk veigrep"])
    for sentence in predictions:
        print(sentence)


df = pd.DataFrame.from_dict(results)



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

bert-base-multilingual-cased Done training


Running Evaluation: 100%|██████████| 192/192 [00:12<00:00, 15.80it/s]


bert-base-multilingual-cased eval_loss 0.2119008700331809
bert-base-multilingual-cased precision 0.37882096069869
bert-base-multilingual-cased recall 0.3956670467502851
bert-base-multilingual-cased f1_score 0.38706079196876747
bert-base-multilingual-cased transformer_model bert-base-multilingual-cased


Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 33.66it/s]


...

ltgoslo/norbert Done training


Running Evaluation: 100%|██████████| 192/192 [00:11<00:00, 16.24it/s]


ltgoslo/norbert eval_loss nan
ltgoslo/norbert precision 0.0
ltgoslo/norbert recall 0.0
ltgoslo/norbert f1_score 0.0
ltgoslo/norbert transformer_model ltgoslo/norbert


Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 35.13it/s]


[{'Mannen': 'I-targ-negative'}, {'på': 'I-targ-negative'}, {'scenen': 'I-targ-negative'}, {'synger': 'I-targ-negative'}, {'stygt': 'I-targ-negative'}]
[{'Damen': 'I-targ-negative'}, {'på': 'I-targ-negative'}, {'scenen': 'I-targ-negative'}, {'synger': 'I-targ-negative'}, {'stygt': 'I-targ-negative'}]
[{'Disse': 'I-targ-negative'}, {'bilene': 'I-targ-negative'}, {'har': 'I-targ-negative'}, {'et': 'I-targ-negative'}, {'fantastisk': 'I-targ-negative'}, {'veigrep': 'I-targ-negative'}]


In [10]:
# Run all above
df

,eval_loss,precision,recall,f1_score,transformer_model
0,0.211901,0.378821,0.395667,0.387061,bert-base-multilingual-cased
1,0.183850,0.525687,0.501710,0.513419,NbAiLab/nb-bert-base
2,NaN,0.000000,0.000000,0.000000,ltgoslo/norbert


In [11]:
df.to_csv(f"summaries/norec_3berts_gpu.csv")